<a href="https://colab.research.google.com/github/YinYinTan/CIND820FinalProject/blob/main/1_Data_Transformation_%26_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

!pip3 install pandas_profiling --upgrade

# Data Transformation

In [93]:
#Read file
df = pd.read_csv('HS.csv', index_col=None)

In [94]:
#Drop rows containing observations that are not valid for the project
df.drop(df[df['EMP_30'] != 1].index, inplace = True)
df.drop(df[df['GEN_10'] == 9].index, inplace = True)
df.drop(df[df['GEN_15'] == 9].index, inplace = True)

In [95]:
#Create new derived variable, which will be the target variable for this project
df['Worse_MH'] = np.where((df['GEN_10'] >= 4) & (df['GEN_15'] >= 4), 0, 1)

In [96]:
#Drop columns pertaining to variables that are not needed for the project
df = df.drop(columns=['PUMFID','VERDATE','EMP_05','BMF_P','EMP_30','ENV_25A','ENV_25B','ENV_25C','GEN_10','GEN_15'])

#Drop columns pertaining to derived variables that are not needed for the project
df = df.drop(columns=['PPEDVEY1','PPEDVEY2','PPEDVFV1','PPEDVFV2','PPEDVGL1','PPEDVGL2','PPEDVGN1','PPEDVGN2','PPEDVHS1','PPEDVHS2','PPEDVMA1','PPEDVMA2','PPEDVOT1','PPEDVOT2','PPEDVRE1','PPEDVRE2','PPEDVRS1','PPEDVRS2','GENDVHDI','GENDVMHI'])

In [97]:
#Reassignment of "Valid Skip" class, for variables pertaining to questions that were not asked of respondents based on their previous responses

#If ENV_30 = “Valid Skip” = 6, then reassign to “No” = 2
df['ENV_30'].mask(df['ENV_30'] == 6, 2, inplace=True)

#If PPE_10 = “No” = 2, then PPE_15A, PPE_15B, PPE_15C, PPE_15D, PPE_15E, PPE_15F, PPE_15G, PPE_15H, PPE_15I, and PPE_15J = “No” = 2 (instead of "Valid Skip")
skip_set_1 = ['PPE_15A','PPE_15B','PPE_15C','PPE_15D','PPE_15E','PPE_15F','PPE_15G','PPE_15H','PPE_15I','PPE_15J']
df[skip_set_1] = np.where(df[['PPE_10']] == 2, 2, df[skip_set_1])

#If PPE_20 = “No” = 2, then PPE_30A, PPE_30B, PPE_30C, PPE_30D, PPE_30E, PPE_30F, PPE_30G, PPE_30H,...
#PPE_35A, PPE_35B, PPE_35C, PPE_35D, PPE_35E, PPE_35F, PPE_35G, PPE_35H = "Not needed for job" = 1 (instead of "Valid Skip")
skip_set_2 = ['PPE_30A','PPE_30B','PPE_30C','PPE_30D','PPE_30E','PPE_30F','PPE_30G','PPE_30H','PPE_35A','PPE_35B','PPE_35C','PPE_35D','PPE_35E','PPE_35F','PPE_35G','PPE_35H']
df[skip_set_2] = np.where(df[['PPE_20']] == 2, 1, df[skip_set_2])

#If PPE_20 = “No” = 2, then PPE_25, PPE_40A, PPE_40B, PPE_40C, PPE_40D, PPE_40E, PPE_40F, PPE_40G, PPE_40H, PPE_40I,...
#PPE_45A, PPE_45B, PPE_45C, PPE_45D, PPE_45E, PPE_45F, PPE_45G, PPE_45H, PPE_45I = "No" = 2 (instead of "Valid Skip")
skip_set_3 = ['PPE_25','PPE_40A','PPE_40B','PPE_40C','PPE_40D','PPE_40E','PPE_40F','PPE_40G','PPE_40H','PPE_40I','PPE_45A','PPE_45B','PPE_45C','PPE_45D','PPE_45E','PPE_45F','PPE_45G','PPE_45H','PPE_45I']
df[skip_set_3] = np.where(df[['PPE_20']] == 2, 2, df[skip_set_3])

In [98]:
#Reassignment of "Not stated" class (i.e. missing value category), for variables pertaining to questions that respondents did not answer

#If response to PPE access question = “Not stated” = 9, then update to "Not needed for job" = 1
missing_set_1 = ['PPE_30A','PPE_30B','PPE_30C','PPE_30D','PPE_30E','PPE_30F','PPE_30G','PPE_30H','PPE_35A','PPE_35B','PPE_35C','PPE_35D','PPE_35E','PPE_35F','PPE_35G','PPE_35H']
for col in missing_set_1:
  df[col].mask(df[col] == 9, 1, inplace=True)

#If response to PPE restriction question = “Not stated” = 9, then update to "No" = 2
missing_set_2 = ['PPE_40A','PPE_40B','PPE_40C','PPE_40D','PPE_40E','PPE_40F','PPE_40G','PPE_40H','PPE_40I','PPE_45A','PPE_45B','PPE_45C','PPE_45D','PPE_45E','PPE_45F','PPE_45G','PPE_45H','PPE_45I']
for col in missing_set_2:
  df[col].mask(df[col] == 9, 2, inplace=True)

#If response to PPE or IPC protocol/practice question = “Not stated” = 9, then remove from dataset
df.drop(df[(df['PPE_05'] == 9)|(df['PPE_10'] == 9)|(df['PPE_15A'] == 9)|(df['PPE_15B'] == 9)|(df['PPE_15C'] == 9)|(df['PPE_15D'] == 9)|(df['PPE_15E'] == 9)|(df['PPE_15F'] == 9)|(df['PPE_15G'] == 9)|(df['PPE_15H'] == 9)|(df['PPE_15I'] == 9)|(df['PPE_15J'] == 9)].index, inplace = True)
df.drop(df[(df['PPE_20'] == 9)|(df['PPE_25'] == 9)].index, inplace = True)
df.drop(df[(df['PPE_50A'] >= 96)|(df['PPE_50B'] >= 96)|(df['PPE_50C'] >= 96)|(df['PPE_50D'] >= 96)|(df['PPE_50E'] >= 96)|(df['PPE_50F'] >= 96)].index, inplace = True)

#Handling of 15/16 remaining variables with classes of "Not stated" = 9: Reassign to most frequent value
missing_set_3 = ['EMP_10','EMP_35','EMP_45','EMPDVGOC','ENV_30','ENVDVCON','ENVDVTYP','ENVDVGRW','GEN_05','GEN_20','AGEDVG4','GDRDVGRP','ISDVFLAG','PGDVFLA','IMMDVGST']
for col in missing_set_3:
  frequent = df[col].mode()
  df[col].mask(df[col] == 9, frequent[0], inplace=True)

In [99]:
#One-hot key encoding of 5 nominal categorical variables
nom_cols = ['GEODVGPR','EMPDVGOC','EMPDVGYW','ENVDVTYP','ENVDVGRW']
df = pd.get_dummies(df, columns=nom_cols, prefix = nom_cols)

In [100]:
#Change all variables to categorical or else they will be considered numeric
df = df.astype('category')

In [101]:
#View basic summary of variables
df.describe()

,EMP_10,EMP_35,EMP_45,ENV_30,ENVDVCON,PPE_05,PPE_10,PPE_15A,PPE_15B,PPE_15C,PPE_15D,PPE_15E,PPE_15F,PPE_15G,PPE_15H,PPE_15I,PPE_15J,PPE_20,PPE_25,PPE_30A,PPE_30B,PPE_30C,PPE_30D,PPE_30E,PPE_30F,PPE_30G,PPE_30H,PPE_35A,PPE_35B,PPE_35C,PPE_35D,PPE_35E,PPE_35F,PPE_35G,PPE_35H,PPE_40A,PPE_40B,PPE_40C,PPE_40D,PPE_40E,...,GDRDVGRP,ISDVFLAG,PGDVFLA,IMMDVGST,Worse_MH,GEODVGPR_10,GEODVGPR_20,GEODVGPR_30,GEODVGPR_45,GEODVGPR_48,GEODVGPR_50,GEODVGPR_60,EMPDVGOC_1,EMPDVGOC_2,EMPDVGOC_3,EMPDVGOC_4,EMPDVGOC_5,EMPDVGOC_6,EMPDVGOC_7,EMPDVGOC_8,EMPDVGOC_99,EMPDVGYW_1,EMPDVGYW_2,EMPDVGYW_3,EMPDVGYW_9,ENVDVTYP_1,ENVDVTYP_2,ENVDVTYP_3,ENVDVTYP_4,ENVDVTYP_5,ENVDVTYP_6,ENVDVTYP_99,ENVDVGRW_10,ENVDVGRW_20,ENVDVGRW_30,ENVDVGRW_45,ENVDVGRW_48,ENVDVGRW_50,ENVDVGRW_60,ENVDVGRW_99
count,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,...,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319
unique,2,2,3,2,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,2,1,1,2,2,1,1,1,1,1,1,1,1,1,1,2,1,1,1,2,2,1,2,2,2,2,1,2,2,1,2,2,2,2,1,2,2,2,2,2,...,2,2,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
freq,14136,12883,12636,12222,9812,14572,16534,15527,15123,13247,15136,13074,13836,10252,12874,14392,9437,16810,13718,13394,14131,7234,9984,9790,9540,13783,13166,13787,15001,7273,10769,10757,10237,14354,12555,9617,10148,14165,12502,16990,...,15316,17065,15603,15476,11929,14834,16205,9421,15690,14817,15718,17229,16774,12120,16717,17057,9828,15821,17051,15885,17299,11658,12969,12425,14905,10060,15628,12203,16697,17220,15303,16803,14846,16263,9440,15697,14829,15723,17229,17206


In [102]:
#Save dataframe to new csv file
df.to_csv('2. Working Dataset.csv', index=False)

# Exploratory Data Analysis (EDA)

In [103]:
profile = ProfileReport(df)
profile.to_file(output_file="3. Dataset Profile.html")

/usr/local/lib/python3.7/dist-packages/pandas_profiling/model/pandas/dataframe_pandas.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df = df.reset_index()


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1442: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return df.reset_index(level=level, drop=drop)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]